In [1]:
!pip install natsort

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from torch.optim.lr_scheduler import StepLR
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
import os
import natsort
from torch.utils import data
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import random
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Test Set

In [3]:
test_names = pd.read_csv('../input/testtxt/test.txt', header = None)

In [4]:
test_ids = pd.DataFrame(test_names[0].str.slice(start = 5, stop = -4)).rename(columns = {0:'ids'}).copy()
test_ids

,ids
0,1.2.826.0.1.3680043.8.498.10003659706701445041...
1,1.2.826.0.1.3680043.8.498.10003890246067211044...
2,1.2.826.0.1.3680043.8.498.10008058926714428108...
3,1.2.826.0.1.3680043.8.498.10008122048552553565...
4,1.2.826.0.1.3680043.8.498.10010309624955962953...
...,...
3577,1.2.826.0.1.3680043.8.498.99840913070180372408...
3578,1.2.826.0.1.3680043.8.498.99893249190615412121...
3579,1.2.826.0.1.3680043.8.498.99930578228271951378...
3580,1.2.826.0.1.3680043.8.498.99953100661957840394...


In [5]:
#create fake labels. we do this just so we can use the CustomDataset class
test = pd.concat([test_ids, pd.DataFrame(np.zeros((len(test_ids),11)))], axis=1)
test

,ids,0,1,2,3,4,5,6,7,8,9,10
0,1.2.826.0.1.3680043.8.498.10003659706701445041...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.2.826.0.1.3680043.8.498.10003890246067211044...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.2.826.0.1.3680043.8.498.10008058926714428108...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.2.826.0.1.3680043.8.498.10008122048552553565...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.2.826.0.1.3680043.8.498.10010309624955962953...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.99840913070180372408...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3578,1.2.826.0.1.3680043.8.498.99893249190615412121...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3579,1.2.826.0.1.3680043.8.498.99930578228271951378...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3580,1.2.826.0.1.3680043.8.498.99953100661957840394...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
fake_test_labels = {}
for i in range(len(test)):
    k = test.iloc[i]['ids']
    v = torch.Tensor(test.iloc[0][1:-1])
    fake_test_labels[k] = v

In [7]:
fake_test_labels

{'1.2.826.0.1.3680043.8.498.10003659706701445041816900371598078663': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10003890246067211044742686138544513464': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10008058926714428108535101002094531530': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10008122048552553565774139897939785611': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10010309624955962953138679469906703941': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10016971606732284006071899723342963784': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10020268825604182610368224629707426511': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10023042737818625910026668901358652653': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10023687805200508996

In [9]:
images_test = CustomDataSet('../input/ranzcr-clip-catheter-line-classification/test',test_transform, fake_test_labels)

In [10]:
len(images_test)

3582

In [11]:
batch_size = 256
test_loader = torch.utils.data.DataLoader(images_test, batch_size=batch_size,
                                                num_workers = 4)

## Load Model

In [12]:
model_file = '../input/model-state-dec2/model_state.pt'

In [15]:
model = SimpleConvnet(1,11)
model.load_state_dict(torch.load(model_file))
model = model.to(device)

In [18]:
preds_by_batch = {}
with torch.no_grad(): 
    for i, samples in enumerate(test_loader):
        # TODO: Put the inputs and targets on the write device
        data, _ = samples
        #target = target.long()
        data = data.to(device)

        # TODO: Feed forward to get the logits
        y_pred_test = model(data)
        preds_by_batch[i] = y_pred_test
        print('Predicted on batch {0}!'.format(i))

Predicted on batch 0!
Predicted on batch 1!
Predicted on batch 2!
Predicted on batch 3!
Predicted on batch 4!
Predicted on batch 5!
Predicted on batch 6!
Predicted on batch 7!
Predicted on batch 8!
Predicted on batch 9!
Predicted on batch 10!
Predicted on batch 11!
Predicted on batch 12!
Predicted on batch 13!


In [22]:
preds_by_batch.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])

In [23]:
preds_by_batch[1]

tensor([[0.0178, 0.0118, 0.0160,  ..., 0.0288, 0.0385, 0.0916],
        [0.0133, 0.0127, 0.0147,  ..., 0.0086, 0.0186, 0.0064],
        [0.0113, 0.0119, 0.0166,  ..., 0.0134, 0.0138, 0.0220],
        ...,
        [0.0204, 0.0098, 0.0287,  ..., 0.0097, 0.0234, 0.0299],
        [0.0270, 0.0181, 0.0172,  ..., 0.0096, 0.0152, 0.0332],
        [0.0192, 0.0055, 0.0119,  ..., 0.0025, 0.0092, 0.0185]],
       device='cuda:0')

In [50]:
tens = []
for i in range(len(preds_by_batch)):
    tens.append(np.array(preds_by_batch[i].cpu()))

In [61]:
preds = pd.DataFrame(np.concatenate(np.array(tens)))
preds['ids'] = test['ids'].copy()
preds = preds[['ids', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
preds = preds.rename(columns = {'ids':'StudyInstanceUID',
                        0:'ETT - Abnormal',
                        1:'ETT - Borderline',
                        2:'ETT - Normal',
                        3:'NGT - Abnormal',
                        4:'NGT - Borderline',
                        5:'NGT - Incompletely Imaged',
                        6:'NGT - Normal',
                        7:'CVC - Abnormal',
                        8:'CVC - Borderline',
                        9:'CVC - Normal',
                        10:'Swan Ganz Catheter Present'})
preds

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.10003659706701445041...,0.012567,0.016707,0.049884,0.014597,0.009303,0.027525,0.950085,0.008341,0.017167,0.012479,0.022179
1,1.2.826.0.1.3680043.8.498.10003890246067211044...,0.041994,0.015714,0.005737,0.008960,0.010451,0.007922,0.974969,0.010180,0.006865,0.016863,0.010042
2,1.2.826.0.1.3680043.8.498.10008058926714428108...,0.015798,0.010100,0.006360,0.019193,0.012782,0.011869,0.979649,0.010176,0.004479,0.020989,0.016752
3,1.2.826.0.1.3680043.8.498.10008122048552553565...,0.014828,0.004553,0.015888,0.033288,0.022436,0.033116,0.984971,0.023256,0.006709,0.011625,0.044588
4,1.2.826.0.1.3680043.8.498.10010309624955962953...,0.021709,0.008760,0.038471,0.027505,0.030464,0.012801,0.968865,0.010575,0.013260,0.017485,0.016697
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.99840913070180372408...,0.006513,0.005455,0.012870,0.005220,0.013559,0.006110,0.974147,0.023101,0.006545,0.009762,0.013479
3578,1.2.826.0.1.3680043.8.498.99893249190615412121...,0.025774,0.023618,0.007345,0.011859,0.019962,0.024104,0.990866,0.011505,0.021319,0.007033,0.051494
3579,1.2.826.0.1.3680043.8.498.99930578228271951378...,0.023348,0.034600,0.010125,0.023156,0.023284,0.040388,0.980453,0.034753,0.040532,0.016176,0.056313
3580,1.2.826.0.1.3680043.8.498.99953100661957840394...,0.014146,0.020251,0.004833,0.011779,0.012597,0.011485,0.976114,0.016411,0.010051,0.007692,0.019989


In [52]:
testd

,ids,0,1,2,3,4,5,6,7,8,9,10
0,1.2.826.0.1.3680043.8.498.10003659706701445041...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.2.826.0.1.3680043.8.498.10003890246067211044...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.2.826.0.1.3680043.8.498.10008058926714428108...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.2.826.0.1.3680043.8.498.10008122048552553565...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.2.826.0.1.3680043.8.498.10010309624955962953...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.99840913070180372408...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3578,1.2.826.0.1.3680043.8.498.99893249190615412121...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3579,1.2.826.0.1.3680043.8.498.99930578228271951378...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3580,1.2.826.0.1.3680043.8.498.99953100661957840394...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform, labels):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)
        self.labels = labels

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc)#.convert("RGB")
        tensor_image = self.transform(image)
        key = os.path.basename(img_loc)[:-4]#this is how we get our ids
        #input labels as dictionary with id number as the key
        label = self.labels[key]
        return tensor_image, label

train_transform = transforms.Compose([
     transforms.Resize((48,48)),
     transforms.ToTensor(),
     transforms.Normalize(
         [0.4824],# 0.4824, 0.4824],
         [0.2142]),# 0.2142, 0.2142])
 ])


test_transform = transforms.Compose([
     transforms.Resize((48,48)),
     transforms.ToTensor(),
     transforms.Normalize(
         [0.4824],# 0.4824, 0.4824],
         [0.2142]),# 0.2142, 0.2142])
 ])

class SimpleConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(SimpleConvnet, self).__init__()

        self.input_channels = input_channels
        self.num_classes = num_classes

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels = self.input_channels, out_channels = 64, kernel_size=5, padding=2),
            nn.ReLU()
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size = 8)
        )

        #final linear layer to project into the correct number of classes
        self.fc = nn.Linear(256, 11)
        self.sig = nn.Sigmoid()
    
    def forward(self, x):
       
        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)

        x = self.block4(x)

        x = View((-1,256))(x)
        x = self.fc(x)
        x = self.sig(x)
        output = x
        
        return output
    
    
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)